In [1]:
import cv2
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]
    
    if width is None and height is None:
        return image

    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

In [5]:
import numpy as np
import cv2
import time

image=0
s1=r"C:\Users\91807\Downloads\Webp.net-resizeimage (1).png"
s2=r"C:\Users\91807\Desktop\DSP PROJECT\Prototyping presentation (1).png"
s3=r"C:\Users\91807\Desktop\DSP PROJECT\Prototyping presentation (2).png"
s4=r"C:\Users\91807\Desktop\DSP PROJECT\Prototyping presentation (3).png"
s5=r"C:\Users\91807\Desktop\DSP PROJECT\Prototyping presentation (4).png"
s6=r"C:\Users\91807\Desktop\DSP PROJECT\Prototyping presentation (5).png"

sl_list=[s1,s2,s3,s4,s5,s6]
index=0
image=cv2.imread(s1)

fixed_display=False
moving_display=False
interactive_mode=False


#5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Take video input from webcam
camera = cv2.VideoCapture(0)

while True:   
    image=cv2.imread(sl_list[index])
    (grabbed, frame) = camera.read()
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
    
    if fixed_display or moving_display:                
        # Define the upper and lower boundaries for - Blue
        '''Blue Filter Parameters'''
        blueLower = np.array([100, 80, 80])
        blueUpper = np.array([140, 255, 255])
        
        # Determine which blue pixels boundaries
        '''Blue Filter'''
        blueMask = cv2.inRange(hsv, blueLower, blueUpper)
        
        # Blur the binary image
        '''The kernel slides through the image - in 2D convolution. 
        A pixel in the original image will be considered 1 only if all the pixels under the kernel is 1,
        otherwise it is eroded (made to zero).'''
        blueMask = cv2.erode(blueMask, kernel, iterations=2)
        ''' For noise removal, erosion is followed by opening and dilation. Because, 
        erosion removes white noises, but it also shrinks our object. So we dilate it.
        Since noise is gone, they won't come back, but our object area increases.
        It is also useful in joining broken parts of an object.'''
        blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
        blueMask = cv2.dilate(blueMask, kernel, iterations=1)   
        
        # Find the parent contours
        (_, cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
        
    if fixed_display:
        xr,yr,wr,hr=0,136,198,202
        try:
            image = image_resize(image, height = hr)
            frame[yr:yr+image.shape[0],xr:xr+image.shape[1]]=image
        except:
            cv2.putText(frame, "PLEASE BRING IN FOCUS!!!",
                        (70, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 2, cv2.LINE_AA)            
        frame = cv2.rectangle(frame, (20,51), (140,105), colors[0], -1)
        frame = cv2.rectangle(frame, (150,51), (255,105), colors[1], -1)
        frame = cv2.rectangle(frame, (265,51), (350,105), colors[2], -1)
    
        cv2.putText(frame, "PREV SLIDE", (30, 81), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, "NEXT SLIDE", (160, 81), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, "CLOSE", (278, 81), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        center = None
        # Check to see if any contours were found
        if len(cnts) > 0:
            # Sort the contours and find the largest one -- we
            # will assume this contour correspondes to the area of the blue region
            cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
            # Get the radius of the enclosing circle around the found contour
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            # Draw the circle around the contour
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            # Get the moments to calculate the center of the contour (in this case Circle)
            M = cv2.moments(cnt)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            
            if center[1] <= 115:
                if 40 <= center[0] <= 140: # Prev Slide
                    if index!= -1*len(sl_list):
                        index=index-1
                        time.sleep(1)

                        
                elif 160 <= center[0] <= 255:
                    if index!=len(sl_list)-1:
                        index=index+1# Next Slide
                        time.sleep(1)
                        
                elif 275 <= center[0] <= 370:
                    fixed_display=False #Stops fixed display
                else:
                    pass
    
    if moving_display:
        if len(cnts) > 0:
            # Sort the contours and find the largest one -- we assume this contour correspondes to the area of the blue region.
            cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            image = image_resize(image, height = h)
            try:
                frame[y:y+image. shape[0],x:x+image. shape[1]]=image
            except:
                cv2.putText(frame, "PLEASE BRING IN FOCUS!!!",
                            (70, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame, "NO SLIDES BEING DISPLAYED",
                            (70, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 2, cv2.LINE_AA)
    
    if interactive_mode:
        frame1=frame.copy()
        image=cv2.imread(sl_list[index])
        image = np.asarray(image)
        value = (35, 35)
        blurred = cv2.GaussianBlur(hsv, value,0)
        # Defining lower range for red color detection.
        '''Red Filter Parameters- 1'''
        lower_red = np.array([0,100,60])
        upper_red = np.array([10,255,255])
        mask1 = cv2.inRange(hsv,lower_red,upper_red)

        # Defining upper range for red color detection
        '''Red Filter Parameters- 2'''
        lower_red = np.array([170,80,50])
        upper_red = np.array([180,255,255])
        mask2 = cv2.inRange(hsv,lower_red,upper_red)

        # Addition of the two masks to generate the final mask.
        mask = mask1+mask2
        '''2D convolution with of mask and kernel'''
        mask = cv2.erode(mask, kernel, iterations=2)
        '''Noise removal'''
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.dilate(mask, kernel, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5),np.uint8))
        
         # Find contours in the image
        (_, cnts, _) = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
        if len(cnts) > 0:
            # Sort the contours and find the largest one -- we assume this contour correspondes to the area of the blue region.
            cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            image = image_resize(image, height = h)
            try:
                frame1[y:y+image. shape[0],x:x+image. shape[1]]=image
                # Replacing red pixels with corresponding pixels of slide image.
                frame[np.where(mask==255)] = frame1[np.where(mask==255)]
                
                
            except:
                cv2.putText(frame, "PLEASE BRING IN FOCUS!!!",
                            (70, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 2, cv2.LINE_AA)
                
        
    

    cv2.imshow("Tracking", frame)
    k = cv2.waitKey(10)
    if k == 27: #esc
        break
    elif k==ord('1'):
        index=0
    elif k==ord('2'):
        index=1
    elif k==ord('3'):
        index=2
    elif k==ord('4'):
        index=3
    elif k==ord('5'):
        index=4
    elif k==ord('6'):
        index=5
    elif k==ord('7'):
        fixed_display=True
    elif k==ord('8'):
        fixed_display=False
    elif k==ord('9'):
        moving_display=True
    elif k==ord('0'):
         moving_display=False
    elif k==ord('i'):
         interactive_mode=True
    elif k==ord('c'):
         interactive_mode=False
    else:
        pass
    
camera.release()
cv2.destroyAllWindows()